In [46]:
!pip install psycopg2 sqlalchemy

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
You should consider upgrading via the '/artifacts/virtualenv/python2/bin/python -m pip install --upgrade pip' command.


<div align="right"><img src="./img/logos.png" alt="drawing" width="400" /> <div/> 



# Data modeling at scale: introduction to dimension modeling
_Diego Ardila - Staff Data Scientist at Shopify_



* [Data modeling at scale?](#data_modeling_at_scale)
  * [What is data warehouse?](#why_a_data_warehouse)
  * [Why a data warehouse?](#what_is_data_warehouse)
  * [What is ETL/ELT?](#what_is_etl)
* [Methodology (dimensional modeling)](#dimensional_modeling)
  * [Example 1: Who are the best movie directors?](#dimensional_modeling_example_1)
    * [Why dimensional modeling](#why_dimensional_modeling_example_1)
    * [Dimensions](#dimensional_modeling_dimensions)
    * [Facts](#dimensional_modeling_facts)
  * [Example 2: Whhen did a show started, and ended? What was the total runtime?](#dimensional_modeling_example_2)

## Data modeling at scale? <a class="anchor" id="data_modeling_at_scale"></a>

*Infrastructure + ETL/ELT + Methodology (dimensional modeling)  = Data warehouse/Data lake*

## What is data warehouse? <a class="anchor" id="what_is_data_warehouse"></a>

1. Database 
2. Data collected from one or multiple sources
3. Designed to support reporting and data analytics

## Why a data warehouse? <a class="anchor" id="why_a_data_warehouse"></a>
1. Keep historic data
2. Central view
3. Data quality and Common model

![alt](https://imgs.xkcd.com/comics/standards_2x.png)

4. Query performance without impacting production systems
5. Augment source system  



## What is ETL/ELT? <a class="anchor" id="what_is_etl"></a>

**Extraction + Transformation + Loading**

- Extract: move data from the source to the data lake. Technlogies such as: [iceberg](https://iceberg.apache.org/), [kafka](https://kafka.apache.org/)  
- Transformation: augment/transform the sources into a consistent format. Technologies such as [spark](https://spark.apache.org/), [dbt](https://www.getdbt.com/), [flink](https://flink.apache.org/), [apache beam](https://beam.apache.org/).   
- Load: load data into frontroom tables for consumption. 

All this typically wrapped around some scheduling solution in batch aplications, e.g. [airflow](https://airflow.apache.org/), [prefect](https://www.prefect.io/), [luigi](https://github.com/spotify/luigi). 

Alternatively, we can talk about ELT

**Extraction + Loading + Transformation**


## Methodology (dimensional modeling) <a class="anchor" id="dimensional_modeling_example_2"></a>
- Data warehouse design technique, developed by [Kimball and Ross](https://www.kimballgroup.com/data-warehouse-business-intelligence-resources/books/data-warehouse-dw-toolkit/). 
- Latest edition (3rd) of the K & R book created in 2013. Right before Redshift and pre-cloud. Still makes sense for understability.
- Based on the "star" model. Two main elements: conformed dimensions and facts:

![alt text](https://cdn.holistics.io/guidebook/star-schema.png "Title")

- Four large steps: 
  - Select the business process
  - Identify grain
  - Identify the dimensions
  - Identify facts
  - Implement model. We can use a SQL transformation engine. Check out [DBT](https://www.getdbt.com/).
- Let's use an example to introduce the main concepts


### Our database: IMDB database (RAW!).




[IMDB datasets](https://www.imdb.com/interfaces/)


<div align="center"><img src="./img/imdb_raw.png" width="500" /> <div/> 

Let's take a look... 

In [7]:
import psycopg2
import sqlalchemy
import urllib
from IPython.display import display

username = 'ssc_workshop'
password = 'sql_for_ds'
host = "ssc-2022-workshop.ct6ghoz7smhy.us-east-1.rds.amazonaws.com"
port = '5432'
engine = sqlalchemy.create_engine('postgresql://{username}:{password}@{host}:{port}/imdb_raw'.format(username=username, password=password, host=host, port=port))


In [8]:
title_basics_sample = pd.read_sql("select * from title_basics limit 10", engine)
crew_sample = pd.read_sql("select * from crew limit 10", engine)
ratings_sample = pd.read_sql("select * from title_ratings limit 10", engine)
episodes_sample = pd.read_sql("select * from episodes limit 10", engine)
print("Ratings sample is...")
display(ratings_sample)
print("Title basics sample...")
display(title_basics_sample)
print("Crew sample is...")
display(crew_sample)
print("Episodes_sample sample is...")
display(episodes_sample)

Ratings sample is...


,index,tconst,averageRating,numVotes
0,0,tt0000001,5.7,1884
1,1,tt0000002,5.9,250
2,2,tt0000003,6.5,1668
3,3,tt0000004,5.8,163
4,4,tt0000005,6.2,2490
5,5,tt0000006,5.2,166
6,6,tt0000007,5.4,774
7,7,tt0000008,5.4,2025
8,8,tt0000009,5.3,195
9,9,tt0000010,6.9,6809


Title basics sample...


,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,19162,tt0019444,movie,Take Me Home,Take Me Home,0,1928.0,None,60.0,"Comedy,Drama,Romance"
1,19163,tt0019445,short,Tall Timber,Tall Timber,0,1928.0,None,8.0,"Animation,Comedy,Short"
2,19164,tt0019446,movie,Der Tanzstudent,Der Tanzstudent,0,1928.0,None,NaN,\N
3,19165,tt0019447,movie,Tarzan the Mighty,Tarzan the Mighty,0,1928.0,None,300.0,"Action,Adventure"
4,19166,tt0019448,movie,Taxi 13,Taxi 13,0,1928.0,None,64.0,"Comedy,Crime,Drama"
5,19167,tt0019449,movie,Tearin' Into Trouble,Tearin' Into Trouble,0,1927.0,None,NaN,Western
6,19168,tt0019450,movie,Telling the World,Telling the World,0,1928.0,None,80.0,"Comedy,Drama"
7,19169,tt0019451,movie,Tempest,Tempest,0,1928.0,None,102.0,"Drama,Romance"
8,19170,tt0019452,movie,Tenderfeet,Tenderfeet,0,1928.0,None,NaN,\N
9,19171,tt0019453,movie,Tenderloin,Tenderloin,0,1928.0,None,85.0,Drama


Crew sample is...


,index,tconst,directors,writers
0,458253,tt0476898,nm0499969,"nm0499969,nm1425526"
1,458254,tt0476899,nm1291436,nm0275425
2,458255,tt0476900,nm1950030,\N
3,458256,tt0476901,nm1950030,\N
4,458257,tt0476902,\N,\N
5,458258,tt0476903,\N,\N
6,458259,tt0476904,\N,\N
7,458260,tt0476905,nm0472016,nm0472016
8,458261,tt0476906,\N,\N
9,458262,tt0476907,\N,\N


Episodes_sample sample is...


,index,tconst,parentTconst,seasonNumber,episodeNumber
0,0,tt0020666,tt15180956,1,2
1,1,tt0020829,tt15180956,1,1
2,2,tt0021166,tt15180956,1,3
3,3,tt0021612,tt15180956,2,2
4,4,tt0021655,tt15180956,2,5
5,5,tt0021663,tt15180956,2,6
6,6,tt0021664,tt15180956,2,4
7,7,tt0021701,tt15180956,2,1
8,8,tt0021802,tt15180956,2,11
9,9,tt0022009,tt15180956,2,10


### Example 1: Who are the best movie directors? <a class="anchor" id="dimensional_modeling_example_1"></a>

#### Why dimensional modeling? <a class="anchor" id="why_dimensional_modeling_example_1"></a>

Let's do first the SQL query....



In [ ]:
who_are_best_movie_directors = """
with directors as (
  select  tconst, d as director
  from crew c, unnest(regexp_split_to_array(directors, ',')) d
),
ratings_directors as (
select d.director, avg(r."averageRating") as averageRating
from 
    title_basics t 
    join directors d on t.tconst = d.tconst
    join title_ratings r on t.tconst = r.tconst
group by 1
)
select rd.director, rd.averageRating
from ratings_directors rd
order by averageRating desc
limit 10 
"""
print(pd.read_sql(who_are_best_movie_directors, engine))

This works, so what are the problems? 

- We carry the naming and standards of the source database
- Okay to perform on ad-hoc basis, but what if you have to regularly use this information?
- Agregations in production are costly. You don't want to do this

Ideally we have something simpler, 

```
select director_id, avg(average_rating) as average_rating
from directors_facts join directors_dimension using(director_id)
group by 1
order by 2 desc
limit 10
```

#### Dimensions <a class="anchor" id="dimensional_modeling_dimensions"></a>

- Business entity. In our example, `directors_dimension`
- There are several types of dimensions:
  - Type 1 dimension
  - Type 2 dimension
- Simple case: type 1 dimension:




In [20]:
create_dimensions = """
drop table if exists directors_dimension_diego_ardila;
create table directors_dimension_diego_ardila as
select d as director_id
from crew c, unnest(regexp_split_to_array(directors, ',')) d
limit 10000;

drop table if exists movie_dimension_diego_ardila;
create table movie_dimension_diego_ardila as
select t.tconst as movie_id, "primaryTitle" as primary_title, "startYear" as release_year, "runtimeMinutes" as runtime_minutes
from title_basics t
limit 10000;

"""
engine.execute(create_dimensions)
print("Movie dimension \n ...")
pd.read_sql("select * from movie_dimension_diego_ardila", engine).head()

Movie dimension


,movie_id,primary_title,release_year,runtime_minutes
0,tt0010047,Daring Hearts,1919.0,NaN
1,tt0010048,A Dark Lantern,1920.0,60.0
2,tt0010049,The Dark Star,1919.0,60.0
3,tt0010050,A Daughter of Eve,1919.0,NaN
4,tt0010051,Daughter of Mine,1919.0,50.0


#### Fact tables <a class="anchor" id="dimensional_modeling_facts"></a>

- Table containing measurements.
- Grain defined by related dimensions.
- Facts are usually additive, but not always.
- Resolves many to many relationships.
- There are several types of fact tables
  - *Transaction Fact Table* represents an event that occurs at the primary point.
  - *Snapshot Fact Table* describes the state of things at a particular time.
  - *Accumulated Fact Table* is used to show the activity of a process that has a beginning and an end. 
- Simple case: Transaction Fact Table of movies directed by a director.



In [15]:
create_facts = """
-- An entry for every movie a director has directed
drop table if exists directors_facts_diego_ardila;
create table directors_facts_diego_ardila as
with directors as (
  select  tconst, d as director_id
  from crew c, unnest(regexp_split_to_array(directors, ',')) d
)
select d.director_id, t.tconst as movie_id, "averageRating" as average_rating, "startYear" as movie_released_on_year
from 
    title_basics t 
    join title_ratings r on t.tconst = r.tconst
    join directors d on t.tconst = d.tconst
limit 100;

"""
engine.execute(create_facts)
print("Directors Facts")
pd.read_sql("select * from directors_facts_diego_ardila", engine).head()

Directors Facts


,director_id,movie_id,average_rating,movie_released_on_year
0,nm0005690,tt0000001,5.7,1894.0
1,nm0721526,tt0000002,5.9,1892.0
2,nm0721526,tt0000003,6.5,1892.0
3,nm0721526,tt0000004,5.8,1892.0
4,nm0005690,tt0000005,6.2,1893.0


How does the query looks now? 


In [19]:
who_are_best_movie_directors_using_dm = """
select director_id, avg(average_rating) as average_rating
from directors_facts_diego_ardila join directors_dimension_diego_ardila using(director_id)
group by 1
order by 2 desc
limit 10 
"""
pd.read_sql(who_are_best_movie_directors_using_dm, engine).head()

,director_id,average_rating
0,nm0718201,7.800000
1,nm0000320,7.433333
2,nm0003593,7.300000
3,nm0210701,7.200000
4,nm0412235,7.000000


##### What if we have a new use case: first year a director directed a movie?



In [ ]:
first_year_director_directed_movie_sql = """
select director_id, min(movie_released_on_year) as first_year_released_movie
from directors_facts_diego_ardila join directors_dimension_diego_ardila using(director_id)
group by 1
order by 2 desc
limit 10 
"""
pd.read_sql(first_year_director_directed_movie_sql, engine).head()

### Example 2: When did a show started and ended? What was the total runtime? <a class="anchor" id="dimensional_modeling_example_2"></a>

1. What grain?: tv show
2. What kind of query are we looking for? 

```
select show_id, primary_title, first_episode_started_on, last_episode_ended_on, number_of_episodes, number_of_seasons, total_run_time
from show_accumulating_facts
```

3. Let's do first the SQL query



In [5]:
when_did_a_show_started_and_ended_sql = """
with series as (
  select tconst, "primaryTitle", "startYear", "titleType"
  from title_basics t
  where "titleType" = 'tvSeries'
),
episodes_series as (
  select tconst, "primaryTitle", "startYear", "runtimeMinutes"
  from title_basics t
  where "titleType" = 'tvSeries'
),
series_facts as (
select s.tconst as series_id
   
    , min(es."startYear") as first_episode_started_on
    , max(es."startYear") as last_episode_ended_on
    , count(distinct "seasonNumber") as number_of_seasons
    , count(distinct "episodeNumber") as number_of_episodes
    , sum(cast(es."runtimeMinutes" as float)) as total_run_time
from 
    series s 
    left join episodes e on s.tconst = e."parentTconst"
    left join episodes_series es on e.tconst = es.tconst
 group by 1
)
select series_id, first_episode_started_on, last_episode_ended_on, number_of_episodes, number_of_seasons, total_run_time
from series_facts
limit 5
"""
pd.read_sql(when_did_a_show_started_and_ended_sql, engine)

,series_id,first_episode_started_on,last_episode_ended_on,number_of_episodes,number_of_seasons,total_run_time
0,tt0025509,None,None,3,1,None
1,tt0035599,None,None,0,0,None
2,tt0035803,None,None,0,0,None
3,tt0038276,None,None,2,2,None
4,tt0039120,None,None,2,2,None


4. Enters dimensional modeling....


#### Which dimensions?

In [12]:
create_dimensions = """
drop table if exists series_dimension_diego_ardila;
create table series_dimension_diego_ardila as
select tconst as series_id, "primaryTitle" as primary_title, "startYear" as series_started_on
from title_basics t
where "titleType" = 'tvSeries'
"""
engine.execute(create_dimensions)
pd.read_sql("select * from series_dimension_diego_ardila limit 100", engine).head()

,series_id,primary_title,series_started_on
0,tt0115124,Buzzkill,1996.0
1,tt0118330,Gun,1997.0
2,tt0025509,Les Misérables,1934.0
3,tt0035599,Voice of Firestone Televues,1943.0
4,tt0035803,The German Weekly Review,1940.0


#### Which facts?

In [10]:
create_facts =  """
drop table if exists series_accumulating_facts_diego_ardila;
create table series_accumulating_facts_diego_ardila as
with series as (
  select tconst, "primaryTitle", "startYear", "titleType"
  from title_basics t
  where "titleType" = 'tvSeries'
),
episodes_series as (
  select tconst, "primaryTitle", "startYear", "runtimeMinutes"
  from title_basics t
  where "titleType" = 'tvSeries'
),
series_facts as (
select s.tconst as series_id 
    , min(es."startYear") as first_episode_started_on
    , max(es."startYear") as last_episode_ended_on
    , count(distinct "seasonNumber") as number_of_seasons
    , count(distinct "episodeNumber") as number_of_episodes
    , sum(cast(es."runtimeMinutes" as float)) as total_run_time
from 
    series s 
    left join episodes e on s.tconst = e."parentTconst"
    left join episodes_series es on e.tconst = es.tconst
 group by 1
)
select series_id, first_episode_started_on, last_episode_ended_on, number_of_episodes, number_of_seasons, total_run_time
from series_facts limit 10
"""
engine.execute(create_facts)



*How does the query looks now?*

In [14]:

pd.read_sql("select * from series_accumulating_facts_diego_ardila limit 10", engine).head()

,series_id,first_episode_started_on,last_episode_ended_on,number_of_episodes,number_of_seasons,total_run_time
0,tt0025509,None,None,3,1,None
1,tt0035599,None,None,0,0,None
2,tt0035803,None,None,0,0,None
3,tt0038276,None,None,2,2,None
4,tt0039120,None,None,2,2,None


### Example 3: What was the first and last shows of a director? What was the best and worst shows? How many shows has a director had?  <a class="anchor" id="dimensional_modeling_example_3"></a>

1. What grain?: tv series 
2. What kind of query are we looking for?

```
select director_id
    , first_series_started_on
    , last_series_started_on
    , fsd.series_id as best_series_id
    , lsd.series_id as last_series_id
    , number_of_series
from directors_accumulating_facts d
    join series_dimension fsd on  d.best_series_id = fsd.series_id
    join series_dimension lsd on  d.last_series_id = lsd.series_id
```
3. Let's do first the SQL query 
4. Which dimensions? Which facts?



# Other topics that we did not covered


- Slow changing dimensions
- Junk dimensions
- Outrigger dimension
- Role playing dimensions
- Bus Architecture
- ... :)  

**Thanks for joining and ...**
<div align="center"><img src="./img/thats_all_folks.png" width="500" /> <div/> 